In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter
import seaborn as sns
if '..' not in sys.path:
    sys.path.append('..')
from run_AC_analysis import compute_fourier_coeffs
from pfcommon import correct_traces

fontsize = 9
lw = 0.75
matplotlib.rc('font', **{'family': 'Arial', 'size': fontsize})
matplotlib.rc('axes', **{'linewidth': 0.75, 'labelsize': fontsize})
matplotlib.rc('xtick', **{'labelsize': fontsize})
matplotlib.rc('ytick', **{'labelsize': fontsize})
matplotlib.rc('xtick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.major', **{'width': lw, 'size':3})
matplotlib.rc('ytick.minor', **{'width': lw, 'size':1.5})

%matplotlib inline

In [ ]:
data_dir = '..'
project_name = 'V2020_Rete_Sardegna_2021_06_03cr_AC'
load_name = 'EqX_MIMC_I2201TR1'
F_start = 10**(-2.5)
F_stop = 1e1
steps_per_decade = 20
dP = 0.1
suffixes = '', #'_FSACTI0201GGR3_OFF', '_SULCTI0202GGR2_OFF'
data_files = ['{}_{}{}_{:.1f}_{:.1f}_{}_dP={:.1f}.npz'.format(project_name, load_name, sfx, np.log10(F_start),
                                                              np.log10(F_stop), steps_per_decade, dP) for sfx in suffixes]
n_files = len(data_files)
blobs = [np.load(os.path.join(data_dir, f), allow_pickle=True) for f in data_files]
F = [b['F'] for b in blobs]
time = [b['time'] for b in blobs]
gamma_c,gamma_s = [], []
config = blobs[0]['config'].item()
for b in blobs:
    freq = b['F']
    time = b['time']
    speed = np.array([d['gen']['s:xspeed'] for d in b['data']], dtype=object)
    γc,γs = compute_fourier_coeffs(freq, time, speed, mu=config['n_periods'])
#     fe = np.array([d['bus']['m:fe'] for d in b['data']], dtype=object)
#     fi = np.array([d['gen']['c:fi'] for d in b['data']], dtype=object)
#     Vr = np.array([d['bus']['m:ur'] for d in b['data']], dtype=object)
#     Vi = np.array([d['bus']['m:ui'] for d in b['data']], dtype=object)
#     V = np.array([np.sqrt(vr**2+vi**2) for vr,vi in zip(Vr,Vi)], dtype=object)
#     γc,γs = compute_fourier_coeffs(freq, time, V, mu=config['n_periods'])
    gamma_c.append(γc)
    gamma_s.append(γs)

In [ ]:
H = [blob['inertia'].item() for blob in blobs]
E = [blob['energy'].item() for blob in blobs]
M = [blob['momentum'].item() for blob in blobs]
print(' Inertia: {} s'.format(' '.join(map(lambda x: f'{x:5.2f}', H))))
print('  Energy: {} MJ'.format(' '.join(map(lambda x: f'{x:5.0f}', E))))
print('Momentum: {} MJs'.format(' '.join(map(lambda x: f'{x:5.0f}', M))))

In [ ]:
attrs = blobs[0]['attributes'].item()['gen']
names = [name.split('__')[0] for name in attrs['loc_name']]
df = pd.DataFrame(data={'H [s]': attrs['typ_id.h'], 'S [MVA]': attrs['typ_id.sgn']}, index=names)
df.to_excel('Sardinia_SM.xlsx')

In [ ]:
dfs = {'gamma_c': [], 'gamma_s': []}
for blob,f,γc,γs in zip(blobs, F, gamma_c, gamma_s):
    gen_names = [name.split('__')[0] for name in blob['device_names'].item()['gen']]
    dfs['gamma_c'].append(pd.DataFrame(data=γc, index=f, columns=gen_names))
    dfs['gamma_s'].append(pd.DataFrame(data=γs, index=f, columns=gen_names))
for key,df in dfs.items():
    with pd.ExcelWriter(key + '.xlsx') as writer:
        for i,mom in enumerate(M):
            df[i].to_excel(writer, sheet_name=f'M={mom:.2f}MJs')

In [ ]:
i = 0
freq,γc,γs = F[i], gamma_c[i], gamma_s[i]
coeffs = [γc - 1j*γs, -γs + 1j*γc, -γc + 1j*γs]
mod,phi = [np.abs(c) for c in coeffs], [np.angle(c) for c in coeffs]
# mod.append(np.sqrt(γc**2+γs**2))
# phi.append(-np.arctan2(γs, γc))

In [ ]:
# %matplotlib notebook
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(7,5))
cmap = 'krg'
ticks = np.array([0.001, 0.01, 0.1, 1, 10, 100])
for i,(γc,γs) in enumerate(zip(gamma_c,gamma_s)):
    for j in range(γc.shape[1]):
#         y = 20 * np.log10(np.sqrt(γc[:,j]**2 + γs[:,j]**2))
        mod = np.sqrt(γc[:,j]**2 + γs[:,j]**2)
        phi = -np.arctan2(γs[:,j], γc[:,j])
        ax[0].plot(F[i], mod, color=cmap[i], lw=0.5, label='M = {:.0f} MJs'.format(M[i]) if j==0 else '')
        ax[1].plot(F[i], phi, color=cmap[i], lw=0.5)
ax[0].legend(loc='upper right', frameon=False, fontsize=8)
for a in ax:
    a.xaxis.set_major_locator(FixedLocator(ticks))
    a.xaxis.set_minor_locator(NullLocator())
    a.set_xlim(ticks[[0,-1]])
    a.set_xscale('log')
ax[1].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
ax[1].set_xlabel('Frequency [Hz]')
ax[0].set_ylabel('Mod')
ax[1].set_ylabel('Φ')
sns.despine()

In [ ]:
%matplotlib inline
rows,cols = 3,6
w,h = 1.5,1.5
device_names = blobs[0]['device_names'].item()['bus']
fig,ax = plt.subplots(rows, cols, figsize=(cols*w, rows*h), sharex=True, sharey=True)
cmap = 'krg'
for i,blob in enumerate(blobs):
    names = blob['device_names'].item()['bus']
    for j,name in enumerate(names):
        try:
            k = device_names.index(name)
            I,J = k//cols, k%cols
            dB = 20 * np.log10(np.sqrt(gamma_c[i][:,j]**2 + gamma_s[i][:,j]**2))
            y = gamma_c[i][:,j]**2 + gamma_s[i][:,j]**2
            ax[I,J].semilogx(F[i], y, color=cmap[i], lw=0.75, label='M = {:.0f} MJs'.format(M[i]))
            ax[I,J].set_title(name.split('___')[0], fontsize=8)
            ax[I,J].xaxis.set_major_locator(FixedLocator(ticks))
            ax[I,J].xaxis.set_minor_locator(NullLocator())
            ax[I,J].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
            ax[I,J].set_xlim(ticks[[0,-1]])
        except:
            print(f'Device name {name} missing')
ax[-1,-1].legend(loc='upper right', frameon=False, fontsize=6)
for a in ax[-1,:]:
    a.set_xlabel('Frequency [Hz]')
# for a in ax[:,0]:
#     a.set_ylabel('[dB20]')
sns.despine()
fig.tight_layout(pad=0.2)
plt.savefig('speed_spectra_AC.pdf')
# plt.savefig('V_spectra_AC.pdf')

In [ ]:
raise Exception('stop here')

In [ ]:
i = 1
ref_gen = 0
δr = fi[i][:, ref_gen]
Vd,Vq = correct_traces(Vr[i], Vi[i], δr)

In [ ]:
j = 5
fig,ax = plt.subplots(3, 2, sharex=True, figsize=(10,4))
ax[0,0].plot(time[0][i], Vd[:,j], 'k', lw=1)
ax[0,1].plot(time[0][i], Vr[i][:,j], 'r', lw=1)
ax[1,0].plot(time[0][i], Vq[:,j], 'k', lw=1)
ax[1,1].plot(time[0][i], Vi[i][:,j], 'r', lw=1)
ax[2,0].plot(time[0][i], np.sqrt(Vd[:,j]**2 + Vq[:,j]**2), 'k', lw=1)
ax[2,1].plot(time[0][i], np.sqrt(Vr[i][:,j]**2 + Vi[i][:,j]**2), 'r', lw=1)
sns.despine()

In [ ]:
fig,ax = plt.subplots(2, 1, figsize=(6,6), sharex=True)
i,j = 0,0
ax[0].plot(time[0][i], fe[i][:,j], 'k', lw=1)
ax[0].plot(time[0][i], speed[i][:,j], 'r', lw=1)
# ax[1].plot(time[0][i], Vr[i][:,j], 'tab:red', lw=1)
# ax[1].plot(time[0][i], Vi[i][:,j], 'tab:green', lw=1)
ax[1].plot(time[0][i], np.sqrt(Vr[i][:,j]**2 + Vi[i][:,j]**2), 'k', lw=1)